NPYファイルを読み込みRVMで線形補間<br>
PTHファイルを読み込みNPYに

In [1]:
import numpy as np
import os
import torch

#DINOの出力PATH
DATASET_PATH = "Crop_Data/256_TRAIN_ver1/TEST256_tuning40_Crop64/Integrated10"

feature_data_pth = torch.load(os.path.join(DATASET_PATH, "trainfeat.pth"))
label_data_pth = torch.load(os.path.join(DATASET_PATH, "trainlabels.pth"))
np.save(os.path.join(DATASET_PATH, "trainfeat.npy"), feature_data_pth)
np.save(os.path.join(DATASET_PATH, "trainlabels.npy"), label_data_pth)

feature_data_npy = np.load(os.path.join(DATASET_PATH, "trainfeat.npy"))
label_data_npy = np.load(os.path.join(DATASET_PATH, "trainlabels.npy"))

print(feature_data_npy.shape)

(643205, 768)


SVR学習

In [3]:
# 必要なライブラリをインポートする
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import notebook

# データを読み込む
feature_data_npy = np.load(os.path.join(DATASET_PATH, "trainfeat.npy"))
label_data_npy = np.load(os.path.join(DATASET_PATH, "trainlabels.npy"))

# データを正規化する（重要な前処理手順）
scaler = StandardScaler()
feature_data_npy = scaler.fit_transform(feature_data_npy)

# 訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(feature_data_npy, label_data_npy, test_size=0.2, random_state=42)

# SVRモデルを作成する
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# プログレスバーを表示しながらモデルを訓練する
num_epochs = 100
for epoch in notebook.tqdm(range(num_epochs), desc="Training"):
    svr_model.fit(X_train, y_train)
    
# テストデータでモデルを評価する
predictions = svr_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Training:   0%|          | 0/100 [00:00<?, ?it/s]